In [1]:
import h5py
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch.utils.data as Data

EPOCH = 1
BATCH_SIZE = 50
LR = 0.001

In [2]:
def loadData():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels
    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels
    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [3]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = loadData()

In [4]:
print("train_set_x_orig shape:", train_set_x_orig.shape)
print("test_set_x_orig shape:", test_set_x_orig.shape)
print("train_set_y_orig shape:", train_set_y_orig.shape)
print("test_set_y_orig shape:", test_set_y_orig.shape)

train_set_x_orig shape: (1080, 64, 64, 3)
test_set_x_orig shape: (120, 64, 64, 3)
train_set_y_orig shape: (1, 1080)
test_set_y_orig shape: (1, 120)


In [5]:
train_set_x_orig = train_set_x_orig.transpose(0, 3, 1, 2)
test_set_x_orig = test_set_x_orig.transpose(0, 3, 1, 2)
print(train_set_x_orig.shape)

(1080, 3, 64, 64)


In [6]:
class subDataset(Data.dataset.Dataset):
    def __init__(self, Data, label):
        self.Data = Data
        self.label = label
    def __len__(self):
        return len(self.Data)
    def __getitem__(self, index):
        data = torch.Tensor(self.Data[index])
        label = torch.Tensor(self.label[index]).long()
        return data, label

In [7]:
dataset = subDataset(train_set_x_orig, train_set_y_orig.T)
test_x = torch.Tensor(test_set_x_orig)
test_y = torch.Tensor(test_set_y_orig.T)
test_y = test_y.squeeze(1)

In [8]:
dataloader = Data.dataloader.DataLoader(dataset = dataset, batch_size = 2, shuffle = True)

In [9]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Sequential(#(3,64,64)
            torch.nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 5, stride = 1, padding = 2),#(6,64,64)
            torch.nn.ReLU(),#(6,64,64)
            torch.nn.MaxPool2d(kernel_size = 2, stride = 2),#(6,32,32)  
        )
        self.conv2 = torch.nn.Sequential(#(6,32,32)  
            torch.nn.Conv2d(6, 16, 5, 1, 2),#(16,32,32)  
            torch.nn.ReLU(),#(16,32,32)  
            torch.nn.MaxPool2d(2, 2)#(16,16,16) 
        )
        self.FC1 = torch.nn.Sequential(
            torch.nn.Linear(16 * 16 * 16, 120),
            torch.nn.ReLU()
        )
        self.FC2 = torch.nn.Sequential(
            torch.nn.Linear(120, 84),
            torch.nn.ReLU()
        )
        self.FC3 = torch.nn.Sequential(
            torch.nn.Linear(84, 10),
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)#(batch,16,16,16)
        x = x.view(x.size(0), -1)
        x = self.FC1(x)
        x = self.FC2(x)
        x = self.FC3(x)

        return x

In [10]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (FC1): Sequential(
    (0): Linear(in_features=4096, out_features=120, bias=True)
    (1): ReLU()
  )
  (FC2): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
  )
  (FC3): Sequential(
    (0): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [11]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = torch.nn.CrossEntropyLoss()

In [16]:
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(dataloader):   # gives batch data, normalize x when iterate train_loader

        output = cnn(b_x)              # cnn output
        loss = loss_func(output, b_y.squeeze(1))   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        
        if step % 50 == 0:
            test_output = cnn(test_x)
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
           
                

Epoch:  0 | train loss: 0.0363 | test accuracy: 0.79
Epoch:  0 | train loss: 0.0061 | test accuracy: 0.77
Epoch:  0 | train loss: 0.0494 | test accuracy: 0.82
Epoch:  0 | train loss: 0.8140 | test accuracy: 0.81
Epoch:  0 | train loss: 0.9992 | test accuracy: 0.80
Epoch:  0 | train loss: 0.1444 | test accuracy: 0.84
Epoch:  0 | train loss: 0.0190 | test accuracy: 0.82
Epoch:  0 | train loss: 0.3147 | test accuracy: 0.74
Epoch:  0 | train loss: 0.0118 | test accuracy: 0.80
Epoch:  0 | train loss: 0.0544 | test accuracy: 0.82
Epoch:  0 | train loss: 0.4165 | test accuracy: 0.83
